# Extract differential expression analsys from Phase 2 Rdata

In [1]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




## Load data

In [2]:
lname = load('/ceph/users/jbenja13/phase3_paper/phase2/_m/deres.Rdata')
lname

[1] "deres"

In [3]:
names(deres)

[1] "development" "region"      "sczd"

## Extract features and select common columns

In [4]:
tx = deres$sczd$tx %>% as.data.frame %>% tibble::rownames_to_column("Feature") %>%
    mutate(Symbol=gene_name, gencodeID=gene_id, gencodeTx=transcript_id)
t_col = colnames(tx)

jxn = deres$sczd$jxn %>% tibble::rownames_to_column("Feature") %>%
    mutate(gencodeID=newGeneID, Symbol=newGeneSymbol)
j_col = colnames(jxn)

gene = deres$sczd$gene %>% as.data.frame %>% tibble::rownames_to_column("Feature")
g_col = colnames(gene)

exon = deres$sczd$exon %>% as.data.frame %>% tibble::rownames_to_column("Feature")
e_col = colnames(exon)

cols = intersect(intersect(intersect(j_col, e_col), g_col), t_col)
cols

Loading required package: S4Vectors

Loading required package: stats4

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply

[1] "Feature"      "ensemblID"    "Symbol"       "gencodeTx"    "meanExprs"   
 [6] "passExprsCut" "logFC"        "AveExpr"      "t"            "P.Value"     
[11] "adj.P.Val"    "B"            "region"       "type"         "gencodeID"

## Merge data

In [5]:
df = bind_rows(gene %>% select(cols, -passExprsCut, -gencodeTx),
               tx %>% select(cols, -passExprsCut, -gencodeTx), 
               exon %>% select(cols, -passExprsCut, -gencodeTx), 
               jxn %>% select(cols, -passExprsCut, -gencodeTx))
df %>% dim

Note: Using an external vector in selections is ambiguous.
ℹ Use `all_of(cols)` instead of `cols` to silence this message.
ℹ See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>.
This message is displayed once per session.



[1] 1622296      13

## Write output

In [6]:
df %>% filter(region == "DLPFC") %>%
    data.table::fwrite("dlpfc_diffExpr_szVctl_full.tsv", sep='\t')

df %>% filter(region == "HIPPO") %>%
    data.table::fwrite("hippo_diffExpr_szVctl_full.tsv", sep='\t')

## Reproducibility Information

In [7]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()

[1] "2021-07-14 13:57:24 EDT"

   user  system elapsed 
 40.930   1.476  38.171 

─ Session info ───────────────────────────────────────────────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 4.0.3 (2020-10-10)
 os       Arch Linux                  
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       America/New_York            
 date     2021-07-14                  

─ Packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────
 package      * version date       lib source        
 assertthat     0.2.1   2019-03-21 [1] CRAN (R 4.0.2)
 base64enc      0.1-3   2015-07-28 [1] CRAN (R 4.0.2)
 BiocGenerics * 0.36.1  2021-04-16 [1] Bioconductor  
 cli            3.0.0   2021-06-30 [1] CRAN (R 4.0.3)
 crayon         1.4.1   2021-02-08 [1] CRAN (R 4.0.3)
 data.table     1.14.0  2021-02-21 [1] CRAN (R 